# Electrolet:  
A Machine Learning Algorithm to Diagnose ECG Arrhythmias Based On the Continuous Wavelet Transform


In [3]:
import wfdb
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import math
from sklearn.utils.class_weight import compute_class_weight
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pywt
import funcs

In [4]:
path = "pickled/"
trainReadings = funcs.unpickler(path + 'train_readings.pkl')
trainDiagnostic = funcs.unpickler(path + 'train_diagnostic.pkl')
validateReadings = funcs.unpickler(path + 'validate_readings.pkl')
validateDiagnostic = funcs.unpickler(path + 'validate_diagnostic.pkl')
testReadings = funcs.unpickler(path + 'test_readings.pkl')
testDiagnostic = funcs.unpickler(path + 'test_diagnostic.pkl')


# Neural network that applies CWT


In [6]:
scaler = StandardScaler()
normalizedTrain = scaler.fit_transform(trainReadings)
normalizedTest = scaler.transform(testReadings)

In [7]:
trainTransform = scaler.fit_transform(funcs.easy_cwt(normalizedTrain, [0.8], 'mexh'))
testTransform = scaler.transform(funcs.easy_cwt(normalizedTest, [0.8], 'mexh'))

In [8]:
# find the standard deviation of the distance between neighboring peaks in the CWT coefficients

trainDeviations = []
for i in trainTransform:
    highest = np.mean(np.array(sorted(i)[-4:-2]))
    peaks = sorted(find_peaks(i, height=highest / 2, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        trainDeviations.append(np.std(np.array(differences)))
    else:
        trainDeviations.append(-1)
        
testDeviations = []
for i in testTransform:
    highest = np.mean(np.array(sorted(i)[-2]))
    peaks = sorted(find_peaks(i, height=highest / 2, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        testDeviations.append(np.std(np.array(differences)))
    else:
        testDeviations.append(-1)
    

In [9]:
zeros = 0
totalZero = 0
ones = 0
totalOne = 0
for i in range(len(trainDiagnostic)):
    if trainDeviations[i] != -1:  # no nan values
        if trainDiagnostic[i] == 0:
            zeros += 1
            totalZero += trainDeviations[i]
        else:
            ones += 1
            totalOne += trainDeviations[i]
        
print("Average standard deviation of length between peaks for sinus rhythm: " + str(totalZero / zeros))
print("Average standard deviation of length between peaks for atrial fibrillation: " + str(totalOne / ones))

Average standard deviation of length between peaks for sinus rhythm: 3.7074495579036832
Average standard deviation of length between peaks for atrial fibrillation: 19.429383041849555


In [10]:
a = []
for i in trainReadings:
    highest = np.mean(np.array(sorted(i)[-7]))
    peaks = sorted(find_peaks(i, height=highest, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        a.append(np.std(np.array(differences)))
    else:
        a.append(-1)

In [ ]:
bump = 10
slideTrainTransform, slideTrainDiagnostic = funcs.make_windows(trainTransform, trainDiagnostic, 500, bump)
slideTestTransform, slideTestDiagnostic = funcs.make_windows(testTransform, testDiagnostic, 500, bump)

In [ ]:
components = 150
pca = PCA(n_components = components) 
pca.fit(slideTrainTransform)
print("PCA represents " + str(sum(pca.explained_variance_ratio_) * 100) + "% of the original variance")

trainComponents = pca.transform(slideTrainTransform).tolist()
testComponents = pca.transform(slideTestTransform).tolist()

In [ ]:
windowsPerOriginal = len(slideTrainTransform) / len(trainTransform)
for i in range(len(trainComponents)):
    trainComponents[i].append(trainDeviations[int(i / windowsPerOriginal)])
    
    
for i in range(len(testComponents)):
    testComponents[i].append(testDeviations[int(i / windowsPerOriginal)])

In [ ]:
trainComponents = np.array(trainComponents)
testComponents = np.array(testComponents)

In [ ]:
def basic_model(activation, shape):
    inputs = Input(shape=shape)
    hidden1 = Dense(20, activation=tf.nn.relu)(inputs)  # amount of neurons in hidden layer is mean between
    # first and last layer
    hidden2 = Dense(20, activation=tf.nn.relu)(hidden1)
    hidden3 = Dense(20, activation=tf.nn.relu)(hidden2)
    hidden4 = Dense(20, activation=tf.nn.relu)(hidden3)
    outputs = Dense(1, activation=activation)(hidden4)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(trainDiagnostic),
                                        y = trainDiagnostic                                                    
                                    )
class_weights = dict(zip(np.unique(trainDiagnostic), class_weights))
class_weights

In [ ]:
modelv3 = basic_model('sigmoid', components + 1)

In [ ]:
modelv3.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', tf.keras.metrics.FalseNegatives()])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)


In [ ]:
modelv3.fit(trainComponents, slideTrainDiagnostic, epochs=100, class_weight=class_weights, callbacks=[callback])

In [ ]:
modelv3.evaluate(testComponents, slideTestDiagnostic)

In [ ]:
funcs.evaluate_model(modelv3.predict(trainComponents), slideTrainDiagnostic)

In [ ]:
windowsPerOriginal = int(len(slideTrainTransform) / len(trainTransform))

predicted = modelv3.predict(testComponents)

slideToRegLabels = []
for i in range(0, len(predicted), windowsPerOriginal):
    current = 0
    for j in range(i, i + windowsPerOriginal):
        current += predicted[j]
    if current >= windowsPerOriginal / 1.2:
        slideToRegLabels.append(1)
    else:
        slideToRegLabels.append(0)

In [ ]:
funcs.evaluate_model(slideToRegLabels, testDiagnostic)

# Neural network that doesn't apply CWT



In [ ]:
trainDeviations_ = []
for i in normalizedTrain:
    highest = np.mean(np.array(sorted(i)[-4:-2]))
    peaks = sorted(find_peaks(i, height=highest / 2, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        trainDeviations_.append(np.std(np.array(differences)))
    else:
        trainDeviations_.append(-1)
        
testDeviations_ = []
for i in normalizedTest:
    highest = np.mean(np.array(sorted(i)[-2]))
    peaks = sorted(find_peaks(i, height=highest / 2, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        testDeviations_.append(np.std(np.array(differences)))
    else:
        testDeviations_.append(-1)

In [ ]:
zeros = 0
totalZero = 0
ones = 0
totalOne = 0
for i in range(len(trainDiagnostic)):
    if trainDeviations[i] != -1:  # no nan values
        if trainDiagnostic[i] == 0:
            zeros += 1
            totalZero += trainDeviations[i]
        else:
            ones += 1
            totalOne += trainDeviations[i]
        
print("Average standard deviation of length between peaks for sinus rhythm: " + str(totalZero / zeros))
print("Average standard deviation of length between peaks for atrial fibrillation: " + str(totalOne / ones))

In [ ]:
a_ = []
for i in trainReadings:
    highest = np.mean(np.array(sorted(i)[-7]))
    peaks = sorted(find_peaks(i, height=highest, distance=50)[0])
    differences = []
    for j in range(1, len(peaks)):
        differences.append(peaks[j] - peaks[j - 1])
    if len(differences) > 0:
        a_.append(np.std(np.array(differences)))
    else:
        a_.append(-1)

In [ ]:
modelv4 = basic_model('sigmoid', components + 1)
modelv4.fit(a, trainDiagnostic)

In [ ]:
funcs.evaluate_model(aTest, testDiagnostic)

In [ ]:
!jupyter nbconvert --to html models.ipynb  


# Convolutional Neural Network